In [1]:
from bs4 import BeautifulSoup as soup
from requests import get
import pandas as pd
import requests
import re
from tqdm import tqdm

In [2]:
url = 'https://www.drdata.in/list-doctors.php?state=Madhya%20Pradesh'
r = requests.get(url)
page_html = soup(r.text, 'html.parser') 
page_html

<!DOCTYPE html>

<html lang="en"><head>
<script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-61981285-1"></script><script>var _$_48c7=["dataLayer","push","js","config","UA-61981285-1"];window[_$_48c7[0]]= window[_$_48c7[0]]|| [];function gtag(){dataLayer[_$_48c7[1]](arguments)}gtag(_$_48c7[2], new Date());gtag(_$_48c7[3],_$_48c7[4])</script><script>(function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
(i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
})(window,document,'script','https://www.google-analytics.com/analytics.js','ga');
ga('create', 'UA-61981285-1', 'auto');
ga('send', 'pageview');</script><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="DrData" name="author"/><meta content="global" name="distribution"/><meta content="

In [3]:
doctors = []
for i in tqdm(range(1,1307)):
  url = 'https://www.drdata.in/list-doctors.php?state=Madhya%20Pradesh&city=Indore&page='+str(i)
  r = requests.get(url)
  page_html = soup(r.text, 'html.parser') 
  table = page_html.find("table",class_="table-bordered table-striped table-condensed cf").find('tbody')
  for i in table.find_all("tr"):
    #print(i)
    name,spec, degree, state, city, details = i.find_all('td')
    doctors.append([name.text, spec.text, degree.text, state.text, city.text, "https://www.drdata.in/"+details.find('a', href=True)['href']])
doctors

100%|██████████| 1306/1306 [29:40<00:00,  1.36s/it]


[['Dr. Aanjne Rajendra Kumar',
  'Oncologist-Radiation',
  'MBBS, MD (Radiation-Oncology)',
  'Madhya Pradesh ',
  'Indore ',
  'https://www.drdata.in/drrajendrakumaraanjne-radiationoncologist'],
 ['Dr. Abbas Shahid',
  'Nephrologist',
  'MBBS, MD DNB (Nephrology)',
  'Madhya Pradesh ',
  'Indore ',
  'https://www.drdata.in/drshahidabbas-nephrologist'],
 ['Dr. Abdul Malik',
  'Allopathic Family Physician',
  'MBBS',
  'Madhya Pradesh ',
  'Indore ',
  'https://www.drdata.in/drmalikabdul-mbbs'],
 ['Dr. Acharya Charulata ',
  'Gynaecologist',
  'MBBS, DGO',
  'Madhya Pradesh ',
  'Indore ',
  'https://www.drdata.in/drcharulataacharya-gynaecologist'],
 ['Dr. Acharya D.C.',
  'AYUSH Doctor',
  'BAMS',
  'Madhya Pradesh ',
  'Indore ',
  'https://www.drdata.in/drdcacharya-ayush'],
 ['Dr. Acharya Dilip',
  'Surgeon (MS General Surgery)',
  'MBBS, MS, FICS, FAIS, FAMS',
  'Madhya Pradesh ',
  'Indore ',
  'https://www.drdata.in/drdilipacharya-surgicalspecialist'],
 ['Dr. Acharya Dinesh',
  'S

In [4]:
df = pd.DataFrame(doctors,columns=["Name", 'Speciality', 'Degree', 'City', 'State', 'Details'])
df

,Name,Speciality,Degree,City,State,Details
0,Dr. Aanjne Rajendra Kumar,Oncologist-Radiation,"MBBS, MD (Radiation-Oncology)",Madhya Pradesh,Indore,https://www.drdata.in/drrajendrakumaraanjne-ra...
1,Dr. Abbas Shahid,Nephrologist,"MBBS, MD DNB (Nephrology)",Madhya Pradesh,Indore,https://www.drdata.in/drshahidabbas-nephrologist
2,Dr. Abdul Malik,Allopathic Family Physician,MBBS,Madhya Pradesh,Indore,https://www.drdata.in/drmalikabdul-mbbs
3,Dr. Acharya Charulata,Gynaecologist,"MBBS, DGO",Madhya Pradesh,Indore,https://www.drdata.in/drcharulataacharya-gynae...
4,Dr. Acharya D.C.,AYUSH Doctor,BAMS,Madhya Pradesh,Indore,https://www.drdata.in/drdcacharya-ayush
...,...,...,...,...,...,...
4268,Mrs. Tripathi Priti,Dietician,MSc (Food & Nutrition),Madhya Pradesh,Indore,https://www.drdata.in/prititripathi-dietician
4269,Mrs. Tripathi Usha,Dietician,MSc (Food & Nutrition),Madhya Pradesh,Indore,https://www.drdata.in/ushatripathi-dietician
4270,Mrs. Vijaywargiya Vidhi,Dietician,MSc (Food & Nutrition),Madhya Pradesh,Indore,https://www.drdata.in/vidhivijaywargiya-dietician
4271,Mrs. Vinita Jaiswal,Dietician,MSc (Foods & Nutrition),Madhya Pradesh,Indore,https://www.drdata.in/vinitajaiswal-dietician


In [5]:
df.to_csv('./Data/doctor_data.csv')

In [2]:
df = pd.read_csv('./Data/doctor_data.csv', index_col=False)
df

,Unnamed: 0,Name,Speciality,Degree,City,State,Details
0,0,Dr. Aanjne Rajendra Kumar,Oncologist-Radiation,"MBBS, MD (Radiation-Oncology)",Madhya Pradesh,Indore,https://www.drdata.in/drrajendrakumaraanjne-ra...
1,1,Dr. Abbas Shahid,Nephrologist,"MBBS, MD DNB (Nephrology)",Madhya Pradesh,Indore,https://www.drdata.in/drshahidabbas-nephrologist
2,2,Dr. Abdul Malik,Allopathic Family Physician,MBBS,Madhya Pradesh,Indore,https://www.drdata.in/drmalikabdul-mbbs
3,3,Dr. Acharya Charulata,Gynaecologist,"MBBS, DGO",Madhya Pradesh,Indore,https://www.drdata.in/drcharulataacharya-gynae...
4,4,Dr. Acharya D.C.,AYUSH Doctor,BAMS,Madhya Pradesh,Indore,https://www.drdata.in/drdcacharya-ayush
...,...,...,...,...,...,...,...
4268,4268,Mrs. Tripathi Priti,Dietician,MSc (Food & Nutrition),Madhya Pradesh,Indore,https://www.drdata.in/prititripathi-dietician
4269,4269,Mrs. Tripathi Usha,Dietician,MSc (Food & Nutrition),Madhya Pradesh,Indore,https://www.drdata.in/ushatripathi-dietician
4270,4270,Mrs. Vijaywargiya Vidhi,Dietician,MSc (Food & Nutrition),Madhya Pradesh,Indore,https://www.drdata.in/vidhivijaywargiya-dietician
4271,4271,Mrs. Vinita Jaiswal,Dietician,MSc (Foods & Nutrition),Madhya Pradesh,Indore,https://www.drdata.in/vinitajaiswal-dietician


In [3]:
invalid = []
doctorDetails = []
for i in tqdm(range(len(df))):
  try:
    url = df['Details'][i]
    #print(url)
    r = requests.get(url)
    page_html = soup(r.text, 'html.parser') 
    table = page_html.find("table",class_="table-bordered table-striped table-condensed cf")
    #print(page_html.find(class_ = 'wp-r'))
    vote = page_html.find('div', {"class" : 'wp-r'}).text.replace(' vote', '') if page_html.find('div', class_ = 'wp-r') else 0
    rating = page_html.find('span', class_ = 'wp-rating-score').text if page_html.find('span', class_ = 'wp-rating-score') else 0
    
    detail = [i.text for i in table.find_all('td')] + ['Rating', rating, 'Votes', vote]
    doctorDetails.append(detail)
  except:
    invalid.append(i)
    pass
doctorDetails

100%|██████████| 4273/4273 [1:15:34<00:00,  1.06s/it]


[['Name',
  'Dr. Aanjne Rajendra Kumar',
  'Specialization',
  'Oncologist-Radiation',
  'Degree',
  'MBBS, MD (Radiation-Oncology)',
  'Area of Practice',
  'Radiation Oncology and Chemotherapy',
  'Clinic/ Hospital Name',
  'Choithram Hospital and Research Centre',
  'Date of Birth',
  '0000-04-16',
  'Address',
  'Institute Of Cancer Sciences Choithram Hospital and Research Centre, Manikbag Road, Indore\nTime-9:00am To 5:00pm\nAdd2-Chothram Clinic, Annapurna Road, Indore\nTime-5:00pm To 7:00pm',
  'State',
  'Madhya Pradesh',
  'District',
  ' Indore',
  'Geographical Area',
  'Manik Bagh Road',
  'About Doctor',
  '\xa0',
  'Rating',
  0,
  'Votes',
  0],
 ['Name',
  'Dr. Abbas Shahid',
  'Specialization',
  'Nephrologist',
  'Degree',
  'MBBS, MD DNB (Nephrology)',
  'Area of Practice',
  'Nephrology, Kidney Specialist',
  'Date of Birth',
  '0000-00-00',
  'Address',
  '21, Babji Nagar, Behind Sagar Automobile, Indore',
  'State',
  'Madhya Pradesh',
  'District',
  ' Indore',
  

In [4]:
len(doctorDetails)

4065

In [5]:
refined_details = []
for i in doctorDetails:
  details = {}
  for j in range(0,len(i),2):
    details[i[j]] = i[j+1]
  refined_details.append(details)
refined_details

[{'About Doctor': '\xa0',
  'Address': 'Institute Of Cancer Sciences Choithram Hospital and Research Centre, Manikbag Road, Indore\nTime-9:00am To 5:00pm\nAdd2-Chothram Clinic, Annapurna Road, Indore\nTime-5:00pm To 7:00pm',
  'Area of Practice': 'Radiation Oncology and Chemotherapy',
  'Clinic/ Hospital Name': 'Choithram Hospital and Research Centre',
  'Date of Birth': '0000-04-16',
  'Degree': 'MBBS, MD (Radiation-Oncology)',
  'District': ' Indore',
  'Geographical Area': 'Manik Bagh Road',
  'Name': 'Dr. Aanjne Rajendra Kumar',
  'Rating': 0,
  'Specialization': 'Oncologist-Radiation',
  'State': 'Madhya Pradesh',
  'Votes': 0},
 {' Member of': '',
  'About Doctor': '\xa0',
  'Address': '21, Babji Nagar, Behind Sagar Automobile, Indore',
  'Area of Practice': 'Nephrology, Kidney Specialist',
  'Date of Birth': '0000-00-00',
  'Degree': 'MBBS, MD DNB (Nephrology)',
  'District': ' Indore',
  'Geographical Area': 'AB Road',
  'Name': 'Dr. Abbas Shahid',
  'Rating': 0,
  'Specializat

In [6]:
df = pd.DataFrame(refined_details)
df

,Name,Specialization,Degree,Area of Practice,Clinic/ Hospital Name,Date of Birth,Address,State,District,Geographical Area,About Doctor,Rating,Votes,Member of,Phone Number,Mobile Number and Email,Address2,Management,Practice as,Website,Graduation Course,Post Graduation Course,Name of Medical Council,Registration Number,Pin Code,Fax Number,After PG Course,Practicing Since
0,Dr. Aanjne Rajendra Kumar,Oncologist-Radiation,"MBBS, MD (Radiation-Oncology)",Radiation Oncology and Chemotherapy,Choithram Hospital and Research Centre,0000-04-16,Institute Of Cancer Sciences Choithram Hospita...,Madhya Pradesh,Indore,Manik Bagh Road,,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dr. Abbas Shahid,Nephrologist,"MBBS, MD DNB (Nephrology)","Nephrology, Kidney Specialist",NaN,0000-00-00,"21, Babji Nagar, Behind Sagar Automobile, Indore",Madhya Pradesh,Indore,AB Road,,0,0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dr. Abdul Malik,Allopathic Family Physician,MBBS,Allopathic Family Medicine,Dr. Abdul Malik Clinic,0000-00-00,"Dr. Abdul Malik Clinic, Patti Bazar, Mhow, Dis...",Madhya Pradesh,Indore,Mhow,,0,0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dr. Acharya Charulata,Gynaecologist,"MBBS, DGO",Gynaecology,"Chaitanya Clinic, Indore",0000-00-00,"Chaitanya Clinic, 42, G/G, Scheme No. 54, Indore",Madhya Pradesh,Indore,Vijay Nagar,,0,0,NaN,0731-2553745,"Send direct SMS and Email to Doctor by using ""...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dr. Acharya D.C.,AYUSH Doctor,BAMS,Ayurved Specialist,NaN,1945-05-21,"39, Sirpur Dhar Road, Indore",Madhya Pradesh,Indore,Dhar Road,,0,0,NaN,0731-2382317,"Send direct SMS and Email to Doctor by using ""...","393-A, Gumasta Nagar, Indore",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4060,Mrs. Tripathi Priti,Dietician,MSc (Food & Nutrition),Food and Nutrition,"LCH Hospital, Indore",0000-00-00,"LCH Hospital, Scheme No.78, Indore",Madhya Pradesh,Indore,Vijay Nagar,,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4061,Mrs. Tripathi Usha,Dietician,MSc (Food & Nutrition),Food and Nutrition,"Totall Diabetes Hormone Institute, Indore",0000-00-00,"Totall Diabetes Hormone Institute, BCM, Health...",Madhya Pradesh,Indore,Vijay Nagar,,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4062,Mrs. Vijaywargiya Vidhi,Dietician,MSc (Food & Nutrition),Food and Nutrition,NaN,0000-00-00,"Vidyapati, Race Course Road, Indore",Madhya Pradesh,Indore,Race Course Road,,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4063,Mrs. Vinita Jaiswal,Dietician,MSc (Foods & Nutrition),Dietician and Nutritionist,"G.K. Hospital, Indore",1978-04-12,"24, Joy Builders Colony, Inside Rani Sati Gate...",Madhya Pradesh,Indore,YN Road,We offer Diet & Nutrition Consultation for Wei...,0,0,INDIAN DIETETIC ASSOCIATION--IAPEN--,NaN,NaN,vinitasjaiswal@gmail.com,NaN,Full Time Employee,http://www.aahaarexpert.com,-B.SC.(FOODS & NUTRITION)---,-M.SC.(FOODS & NUTRITON)---,NaN,NaN,NaN,NaN,-DIABETES EDUCATOR---,NaN


In [7]:
df.isnull().sum()

Name                          0
Specialization                0
Degree                       10
Area of Practice             93
Clinic/ Hospital Name      2173
Date of Birth                 0
Address                       9
State                         0
District                      0
Geographical Area           371
About Doctor                823
Rating                        0
Votes                         0
 Member of                 2443
Phone Number               2341
Mobile Number and Email    2341
Address2                   1473
Management                 3520
Practice as                3633
Website                    3937
Graduation Course          4026
Post Graduation Course     4036
Name of Medical Council    4000
Registration Number        4000
Pin Code                   3242
Fax Number                 4043
After PG Course            4051
 Practicing Since          4062
dtype: int64

In [8]:
df.to_csv('./Data/doctor_details.csv')

In [9]:
df.shape

(4065, 28)